최종 코드

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import json

driver = webdriver.Chrome()

# 1. 포털 로그인
driver.get("https://portal.dankook.ac.kr/p/S01/")
time.sleep(2)

USER_ID = input("학번: ")
USER_PW = input("비밀번호: ")
driver.find_element(By.ID, "user_id").send_keys(USER_ID)
driver.find_element(By.ID, "user_password").send_keys(USER_PW)
driver.find_element(By.ID, "user_password").send_keys(Keys.RETURN)
time.sleep(3)

# 2. VOC 진입
action = ActionChains(driver)
menu = driver.find_element(By.LINK_TEXT, "행정서비스")
action.move_to_element(menu).perform()
time.sleep(1)

voc_link = driver.find_element(By.CSS_SELECTOR, "a[href='https://voc.dankook.ac.kr/tiad/vocm/main.do']")
voc_link.click()
time.sleep(2)

driver.switch_to.window(driver.window_handles[-1])
time.sleep(1)

results = []

# 3. 전체 FAQ 링크 수집
faq_links = []
for page in range(1, 36):
    print(f"📄 {page}페이지 링크 수집 중...")
    driver.get(f"https://voc.dankook.ac.kr/tiad/vocm/findFaqList.do?pagination.currentPageNo={page}")
    time.sleep(2)

    rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")
    for row in rows:
        try:
            tds = row.find_elements(By.TAG_NAME, "td")
            if len(tds) < 4:
                continue

            category = tds[2].text.strip()
            title_text = tds[3].text.strip()
            link = tds[3].find_element(By.TAG_NAME, "a").get_attribute("href")

            faq_links.append({
                "category": category,
                "title": title_text,
                "url": link
            })
        except:
            continue

# 4. 상세 페이지에서 answer 수집
for i, item in enumerate(faq_links):
    print(f"🔍 {i+1}/{len(faq_links)}: {item['title']}")
    driver.get(item["url"])
    time.sleep(1.5)

    if "findFaqDetail" not in driver.current_url:
        print("⚠️ 리다이렉트 발생 → 건너뜀")
        continue

    try:
        answer = ""

        # ① 일반 구조
        paragraphs = driver.find_elements(By.CSS_SELECTOR, "div pre p")
        if len(paragraphs) >= 2:
            answer_lines = [p.text.strip() for p in paragraphs if p.text.strip()]
            answer = "\n".join(answer_lines)

        else:
            # ② td 구조
            rows_detail = driver.find_elements(By.CSS_SELECTOR, ".tbl_view tbody tr")
            answer_lines = []
            for r in rows_detail:
                tds_detail = r.find_elements(By.TAG_NAME, "td")
                if len(tds_detail) >= 2:
                    content = tds_detail[1].text.strip()
                    if content:
                        answer_lines.append(content)
            answer = "\n".join(answer_lines)

        # ③ fallback + 메타 정보 제거
        if not answer.strip():
            fallback = driver.find_element(By.CSS_SELECTOR, ".tbl_view")
            full_text = fallback.text.strip()
            lines = full_text.split("\n")

            # 작성자/작성일/캠퍼스/분류/제목 제거 (대략 앞 5줄)
            if len(lines) > 5:
                answer = "\n".join(lines[5:])
            else:
                answer = full_text

        results.append({
            "category": item["category"],
            "title": item["title"],
            "answer": answer
        })

    except Exception as e:
        print(f"❌ 오류: {e}")
        continue

# 5. JSON 저장
with open("dku_faq.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"\n✅ 크롤링 완료! 총 {len(results)}개 항목 저장됨 → dku_faq.json")


📄 1페이지 링크 수집 중...
📄 2페이지 링크 수집 중...
📄 3페이지 링크 수집 중...
📄 4페이지 링크 수집 중...
📄 5페이지 링크 수집 중...
📄 6페이지 링크 수집 중...
📄 7페이지 링크 수집 중...
📄 8페이지 링크 수집 중...
📄 9페이지 링크 수집 중...
📄 10페이지 링크 수집 중...
📄 11페이지 링크 수집 중...
📄 12페이지 링크 수집 중...
📄 13페이지 링크 수집 중...
📄 14페이지 링크 수집 중...
📄 15페이지 링크 수집 중...
📄 16페이지 링크 수집 중...
📄 17페이지 링크 수집 중...
📄 18페이지 링크 수집 중...
📄 19페이지 링크 수집 중...
📄 20페이지 링크 수집 중...
📄 21페이지 링크 수집 중...
📄 22페이지 링크 수집 중...
📄 23페이지 링크 수집 중...
📄 24페이지 링크 수집 중...
📄 25페이지 링크 수집 중...
📄 26페이지 링크 수집 중...
📄 27페이지 링크 수집 중...
📄 28페이지 링크 수집 중...
📄 29페이지 링크 수집 중...
📄 30페이지 링크 수집 중...
📄 31페이지 링크 수집 중...
📄 32페이지 링크 수집 중...
📄 33페이지 링크 수집 중...
📄 34페이지 링크 수집 중...
📄 35페이지 링크 수집 중...
🔍 1/347: 외국인 유학생(학부, 어학원) 공통 FAQ
🔍 2/347: 국제학생증은 어떻게 발급받을 수 있나요?
🔍 3/347: (공통) 바닥이 더러워 청소를 하고 싶은데 어떻게 하나요? (바닥 왁스 작업)
🔍 4/347: (공통) 교내에 현수막을 설치하고 싶은데 어디서 확인을 받아 설치하나요.
🔍 5/347: (공통) 학과행사 관련 포터(트럭)는 어떻게 대여하여 사용 할 수 있나요?
🔍 6/347: (공통) 교내 강의실 및 시설물의 사용신청은 어떻게 하나요?
🔍 7/347: (공통) 교내에 여학생 샤워실은 없나요?
🔍 8/347: [KIP] Korean Immersion Program은 어떤 프로그램인가

크롤링한 데이터 정리

In [17]:
import json
import re

# 원본 전체 FAQ 로드
with open("dku_faq.json", "r", encoding="utf-8") as f:
    faq_data = json.load(f)

final_results = []

# Q/A 구조 분리 함수
def split_qna(faq):
    qna_list = []
    pattern = re.compile(r'Q\d+\s*\n(.*?)\nA\d+\s*\n(.*?)(?=\nQ\d+|\Z)', re.DOTALL)
    matches = pattern.findall(faq["answer"])
    for q_text, a_text in matches:
        qna_list.append({
            "category": faq["category"],
            "title": q_text.strip(),
            "answer": a_text.strip()
        })
    return qna_list

# 전체 FAQ 순회
for faq in faq_data:
    # Q/A 구조 있는 경우 → 쪼개서 넣음
    split_items = split_qna(faq)
    if split_items:
        final_results.extend(split_items)
    else:
        # Q/A 구조 없는 경우 → 그대로 사용하되 title/answer는 정리
        final_results.append({
            "category": faq["category"],
            "title": faq["title"].strip(),
            "answer": faq["answer"].strip()
        })

# 저장
with open("dku_faq_all.json", "w", encoding="utf-8") as f:
    json.dump(final_results, f, ensure_ascii=False, indent=2)

print(f"✅ 전체 FAQ 처리 완료! 총 {len(final_results)}개 항목 저장됨 → dku_faq_all.json")

✅ 전체 FAQ 처리 완료! 총 384개 항목 저장됨 → dku_faq_all.json
